El otro día estuvimos hablando de la [biblioteca `collections`](http://pybonacci.org/2016/05/08/joyitas-en-la-stdlib-collections/), una joya dentro de la librería estándar. Hoy vamos a hablar de una nueva biblioteca que se incluyó en la versión 3.4 de CPython llamada `pathlib`. Esta biblioteca nos da la posibilidad de usar clases para trabajar con las rutas del sistema de ficheros con una serie de métodos muy interesantes.

## Ejemplo usando lo disponible hasta hace poco

Pensemos en un problema que consiste en identificar todos los ficheros *.py* disponibles en determinada ruta y dejarlos en una nueva carpeta todos juntos sin borrarlos de la carpeta original en la que se encuentren.

Vamos a crear un directorio de prueba que se llamará *pybonacci_probando_pathlib* dentro del cual habrá una serie de carpetas y ficheros:

In [38]:
import os
import glob
import shutil
from random import randint, choice, seed
from string import ascii_letters

seed(1)

base = os.path.join(os.path.curdir,
                    'pybonacci_probando_pathlib')
os.makedirs(base, exist_ok = True)

for i in range(0, randint(3, 5)):
    folder = ''.join([choice(ascii_letters) for _ in range(4)])
    path = os.path.join(base, folder)
    os.makedirs(path, exist_ok = True)
    for j in range(0, randint(2, 5)):
        ext = choice(['.txt', '.py', '.html'])
        name = ''.join([choice(ascii_letters) for _ in range(randint(5, 10))])
        filename = name + ext
        path2 = os.path.join(path, filename)
        open(path2, 'w').close()

Nos debería quedar una estructura parecida a lo siguiente:

    pybonacci_probando_pathlib/
    ├── KZWe
    │   ├── CrUZoLgubb.txt
    │   ├── IayRnBUbHo.txt
    │   ├── WCEPyYng.txt
    │   └── yBMWX.py
    ├── WCFJ
    │   ├── GBGQmtsLFG.html
    │   ├── PglOUshVv.py
    │   └── RoWDsb.py
    └── zLcE
        ├── AQlxJSXR.html
        ├── fCQGgXk.html
        └── xFUbEctT.html

Ok, vamos a mover todo los ficheros *.py* a la carpeta *tmp* dentro de *pybonacci_probando_pathlib* de la forma 'antigua'. ¿Cómo sería esto?

In [39]:
# recolectamos todos los ficheros *.py con sus rutas
ficheros_py = glob.glob(os.path.join(base, '**', '*.py'))

# creamos la carpeta tmp dentro de pybonacci_probando_pathlib
os.makedirs(os.path.join(base, 'tmp'), exist_ok = True)

# y copiamos los ficheros a la nueva carpeta tmp
for f in ficheros_py:
    fich = f.split(os.path.sep)[-1]
    shutil.copyfile(f, os.path.join(base, 'tmp', fich))

En el anterior ejemplo hemos tenido que usar las bibliotecas `glob`, `os` y `shutil` para poder realizar una operación relativamente sencilla.

Esto no es del todo deseable.

Pero, por suerte, disponemos de la nueva biblioteca `pathlib`. Veamos el mismo ejemplo usando esta librería. Primero borramos la carpeta *tmp*.

In [40]:
from pathlib import Path

p = Path('.', 'pybonacci_probando_pathlib', 'tmp')
for i in p.iterdir():
    i.unlink()
p.rmdir()

Lo anterior, con `shutil` sería `shutil.rmtree(path_to_folder_to_remove)`, un poco más sencillo. Esto es lo único que hecho de menos en `pathlib`. Algo negativo tenía que tener.

Ahora veamos cómo realizar lo mismo que antes usando `pathlib`.

In [95]:
# recolectamos todos los ficheros *.py con sus rutas
p = Path('.', 'pybonacci_probando_pathlib')
ficheros_py = list(p.glob('**/*.py'))

# creamos la carpeta tmp dentro de pybonacci_probando_pathlib
(p / 'tmp').mkdir(mode = 0o777, exist_ok = True)

# y copiamos los ficheros a la nueva carpeta tmp
for f in ficheros_py:
    data = f.open().readline()
    with open(str(Path(p, 'tmp', f.parts[-1])), 'w') as f:
        f.write(data)